# Herbie Speedup  

Currently, I'm slowed down by taking the inventory of each grib file and then using regex to find the byte ranges I'm interested in, then the coordinate locations I'm interested in. The whole process takes ~1.5 minutes for the analysis data of a given year... How can I make this faster?

In [1]:
import datetime
import os

from herbie import Herbie
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import pvlib
import pytz

from common import _grib_variables_sfc, san_antonio, get_search_string, get_grib_data

import multiprocessing as mp
mp.cpu_count()

10

## are byte ranges constant for a given FXX? (especially within analysis files?)

In [2]:
search_string = search_string = get_search_string([i.get("searchstring") for i in _grib_variables_sfc.values()])

date1 = datetime.datetime(2022, 1, 1, 0, 0)
date2 = datetime.datetime(2022, 1, 1, 6, 0)
date3 = datetime.datetime(2022, 1, 1, 12, 0)
date4 = datetime.datetime(2022, 1, 1, 18, 0)
date5 = datetime.datetime(2023, 1, 1, 0, 0)
date6 = datetime.datetime(2023, 1, 1, 6, 0)
date7 = datetime.datetime(2023, 1, 1, 12, 0)
date8 = datetime.datetime(2023, 1, 1, 18, 0)
date9 = datetime.datetime(2023, 6, 3, 0, 0)
date10 = datetime.datetime(2023, 5, 17, 6, 0)
date11 = datetime.datetime(2023, 9, 20, 12, 0)
date12 = datetime.datetime(2023, 3, 28, 18, 0)
dates = [date1, date2, date3, date4, date5, date6, date7, date8, date9, date10, date11, date12]

In [3]:
H = Herbie(dates[0],
           product="sfc",
           fxx=0)
H.inventory(search_string)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
4,5,1617526,2972751,1617526-2972751,2022-01-01,2022-01-01,VIS,surface,anl,:VIS:surface:anl
8,9,3697313,4914221,3697313-4914221,2022-01-01,2022-01-01,GUST,surface,anl,:GUST:surface:anl
58,59,27676283,27676471,27676283-27676471,2022-01-01,2022-01-01,LTNG,entire atmosphere,anl,:LTNG:entire atmosphere:anl
61,62,29911092,31426189,29911092-31426189,2022-01-01,2022-01-01,PRES,surface,anl,:PRES:surface:anl
63,64,33584657,34874302,33584657-34874302,2022-01-01,2022-01-01,TMP,surface,anl,:TMP:surface:anl
68,69,37269249,37441836,37269249-37441836,2022-01-01,2022-01-01,SNOWC,surface,anl,:SNOWC:surface:anl
69,70,37441836,38062896,37441836-38062896,2022-01-01,2022-01-01,SNOD,surface,anl,:SNOD:surface:anl
70,71,38062896,39243986,38062896-39243986,2022-01-01,2022-01-01,TMP,2 m above ground,anl,:TMP:2 m above ground:anl
73,74,41725249,42909431,41725249-42909431,2022-01-01,2022-01-01,DPT,2 m above ground,anl,:DPT:2 m above ground:anl
74,75,42909431,44490727,42909431-44490727,2022-01-01,2022-01-01,RH,2 m above ground,anl,:RH:2 m above ground:anl


In [4]:
H = Herbie(dates[1],
           product="sfc",
           fxx=0)
H.inventory(search_string)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jan-01 06:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
4,5,1780102,3134395,1780102-3134395,2022-01-01 06:00:00,2022-01-01 06:00:00,VIS,surface,anl,:VIS:surface:anl
8,9,3920216,5213660,3920216-5213660,2022-01-01 06:00:00,2022-01-01 06:00:00,GUST,surface,anl,:GUST:surface:anl
58,59,28287217,28287405,28287217-28287405,2022-01-01 06:00:00,2022-01-01 06:00:00,LTNG,entire atmosphere,anl,:LTNG:entire atmosphere:anl
61,62,30647628,32162055,30647628-32162055,2022-01-01 06:00:00,2022-01-01 06:00:00,PRES,surface,anl,:PRES:surface:anl
63,64,34320523,35616587,34320523-35616587,2022-01-01 06:00:00,2022-01-01 06:00:00,TMP,surface,anl,:TMP:surface:anl
68,69,38055974,38254846,38055974-38254846,2022-01-01 06:00:00,2022-01-01 06:00:00,SNOWC,surface,anl,:SNOWC:surface:anl
69,70,38254846,38889362,38254846-38889362,2022-01-01 06:00:00,2022-01-01 06:00:00,SNOD,surface,anl,:SNOD:surface:anl
70,71,38889362,40078644,38889362-40078644,2022-01-01 06:00:00,2022-01-01 06:00:00,TMP,2 m above ground,anl,:TMP:2 m above ground:anl
73,74,42607515,43805808,42607515-43805808,2022-01-01 06:00:00,2022-01-01 06:00:00,DPT,2 m above ground,anl,:DPT:2 m above ground:anl
74,75,43805808,45401699,43805808-45401699,2022-01-01 06:00:00,2022-01-01 06:00:00,RH,2 m above ground,anl,:RH:2 m above ground:anl


In [5]:
H = Herbie(dates[4],
           product="sfc",
           fxx=0)
H.inventory(search_string)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
4,5,2010606,3335525,2010606-3335525,2023-01-01,2023-01-01,VIS,surface,anl,:VIS:surface:anl
8,9,4338018,5558586,4338018-5558586,2023-01-01,2023-01-01,GUST,surface,anl,:GUST:surface:anl
58,59,29018540,29018728,29018540-29018728,2023-01-01,2023-01-01,LTNG,entire atmosphere,anl,:LTNG:entire atmosphere:anl
61,62,31277621,32786561,31277621-32786561,2023-01-01,2023-01-01,PRES,surface,anl,:PRES:surface:anl
63,64,34940256,36222333,34940256-36222333,2023-01-01,2023-01-01,TMP,surface,anl,:TMP:surface:anl
68,69,38519476,38663600,38519476-38663600,2023-01-01,2023-01-01,SNOWC,surface,anl,:SNOWC:surface:anl
69,70,38663600,39290506,38663600-39290506,2023-01-01,2023-01-01,SNOD,surface,anl,:SNOD:surface:anl
70,71,39290506,40475141,39290506-40475141,2023-01-01,2023-01-01,TMP,2 m above ground,anl,:TMP:2 m above ground:anl
73,74,43055488,44234631,43055488-44234631,2023-01-01,2023-01-01,DPT,2 m above ground,anl,:DPT:2 m above ground:anl
74,75,44234631,45804244,44234631-45804244,2023-01-01,2023-01-01,RH,2 m above ground,anl,:RH:2 m above ground:anl


Grib byte ranges are not the same across grib files

## Where is time spent?

In [7]:
import logging
logging.basicConfig(
    level=logging.DEBUG, 
    format='%(asctime)s - %(levelname)s - %(message)s', 
    filename="2023.08.03 - Herbie Speedup.log",
    filemode='w'
)

grib_data = get_grib_data(dates[0], "./", search_string, san_antonio.get("longitude"), san_antonio.get("latitude"))


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


👨🏻‍🏭 Created directory: [/Users/jeh/data/hrrr/20220101]
Note: Returning a list of [9] xarray.Datasets because cfgrib opened with multiple hypercubes.


creating the Herbie object takes about 1 second  
x array operation takes 49 seconds (4 seconds of which are after ascii encoding detected)  
2 seconds to iterate through all datasets, locate, and extract variable values

## Invalid latitude error

In [6]:
# H = Herbie(dates[0], product="sfc", fxx=0)
# dss = H.xarray(search_string)
# dss[0]

Also invalid latitude error  
    latitudes 21.14 - 52.6  
    longitudes 225.9 - 299  



# Note that lonestar's execution seems to be stalled on 2022092705... it could be there are some missing files that are being filled in but seems unlikely